# A small convnet from scratch

From: https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

We will use a very small convnet with few layers and few filters per layer, alongside data augmentation and dropout. Dropout also helps reduce overfitting, by preventing a layer from seeing twice the exact same pattern, thus acting in a way analoguous to data augmentation (you could say that both dropout and data augmentation tend to disrupt random correlations occuring in your data).

The code snippet below is our first model, a simple stack of 3 convolution layers with a ReLU activation and followed by max-pooling layers. This is very similar to the architectures that Yann LeCun advocated in the 1990s for image classification (with the exception of ReLU).

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator

Using Theano backend.


In [2]:
from keras import backend as K
K.set_image_dim_ordering('tf')

In [3]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# the model so far outputs 3D feature maps (height, width, features)

On top of it we stick two fully-connected layers. We end the model with a single unit and a sigmoid activation, which is perfect for a binary classification. To go with it we will also use the binary_crossentropy loss to train our model.

In [4]:
model.add(Flatten()) # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [5]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        '../data/dogs_vs_cats/train',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=32,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        '../data/dogs_vs_cats/validation',
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

Found 7222 images belonging to 2 classes.
Found 888 images belonging to 2 classes.


In [6]:
model.fit_generator(
        train_generator,
        samples_per_epoch=2000,
        nb_epoch=10,
        validation_data=validation_generator,
        nb_val_samples=800)
# always save your weights after training or during training
model.save_weights('convnet_from_scratch_weights.h5')  

Epoch 1/10
1984/2000 [============================>.] - ETA: 0s - loss: 0.7346 - acc: 0.5302

/Users/rafael/anaconda/envs/wattx/lib/python3.5/site-packages/keras/engine/training.py:1569: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


2016/2000 [==============================] - 112s - loss: 0.7331 - acc: 0.5337 - val_loss: 0.6535 - val_acc: 0.6188
Epoch 2/10
2016/2000 [==============================] - 120s - loss: 0.6697 - acc: 0.5699 - val_loss: 0.6970 - val_acc: 0.5170
Epoch 3/10
2016/2000 [==============================] - 108s - loss: 0.6525 - acc: 0.6280 - val_loss: 0.6230 - val_acc: 0.6638
Epoch 4/10
2006/2000 [==============================] - 98s - loss: 0.6183 - acc: 0.6615 - val_loss: 0.5528 - val_acc: 0.7233
Epoch 5/10
2016/2000 [==============================] - 103s - loss: 0.6163 - acc: 0.6716 - val_loss: 0.6884 - val_acc: 0.6262
Epoch 6/10
2016/2000 [==============================] - 108s - loss: 0.6023 - acc: 0.6691 - val_loss: 0.5419 - val_acc: 0.7257
Epoch 7/10
2016/2000 [==============================] - 92s - loss: 0.5882 - acc: 0.6860 - val_loss: 0.5488 - val_acc: 0.7172
Epoch 8/10
2006/2000 [==============================] - 91s - loss: 0.5800 - acc: 0.7079 - val_loss: 0.6520 - val_acc: 0.609